![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 1<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

# Aim

This assignment aims to solve a problem using Informed and Uninformed search algorithms such as BTS, IDS, and A* algorithm. Finally, we will compare the time complexity of these algorithms.

# Problem Definition

## Intial State

The initial state is the state that our agent will start from. In this case, Gandalf starts from his state at a given position in the input.

## Action

The agent can do an action to reach another state. In this case, Gandolf can have six actions: going up, right, down, and left and picking a friend, and placing them in another position.

## Transition Model

UP action: If it is permitted, the agent will move from state (x,y) to state (x-1,y)

DOWN action: If it is permitted, the agent will move from state (x,y) to state (x+1,y)

RIGHT action: If it is permitted, the agent will move from state (x,y) to state (x,y+1)

LEFT action: If it is permitted, the agent will move from state (x,y) to state (a,y-1)

PICK action: the agent will pick the friend

PLACE action: the agent will place the friend

## Goal State

The goal state is that the agent has successfully done its tasks. In this case, Gandalf must place all the friends in their goal positions and reach the final point (Gandor).

# Algorithms

## BFS

The Breadth-First Search (BFS) is an algorithm for traversing or searching tree or graph data structures. It explores all the nodes at the present depth before moving to the next depth level. BFS is complete, and it will give us an optimal solution with time complexity of $O(b^d)$.

## IDS

The IDS or Iterative Deepening Search is an algorithm based on DFS which controls the depth before expanding another node. IDS leads to an optimal solution. Unlike DFS, its time complexity is $O(bm)$.

## A*

A* algorithm searches for the shortest path between the initial and the final state. It is used in various applications, such as maps.
In maps, the A* algorithm calculates the shortest distance between the source (initial state) and the destination (final state).
$g(n)$ is the cost of reaching the current state, and $h(n)$ is the heuristic function that estimates the cost of getting to the final state from the current node $n$.
A* search is complete, and its time complexity is $O(b^d)$.

# Classes

In [1]:
import queue
from time import time
import copy

## Node

In [2]:
class Node(object):
    def __init__(self,position):
        self.state=tuple(position)
        self.parent=None
        self.path=''
        self.picked_RF=None
        self.placed_RFs=set()
        self.inOrk=None
        self.presence=0
        self.depth=0
        self.h=0

    def __eq__(self, other):
        if self.state==other.state and self.path==other.path and self.picked_RF==other.picked_RF and self.placed_RFs==other.placed_RFs:
            return True
        return False

    def __hash__(self):
        return hash((self.state,str(self.placed_RFs),self.picked_RF,self.depth))
    


## Problem

The code to read the problem from the test file

In [3]:
test_name='test_03'
orks=[]
RFs_initial_pos=[]
RFs_goal_pos=[]
with open('./tests/'+test_name+'.txt') as f:
    n,m = tuple(map(int,f.readline().split()))
    gandalf_initial_pos = tuple(map(int,f.readline().split()))
    gandor = tuple(map(int,f.readline().split()))
    k,l = tuple(map(int,f.readline().split()))
    for _ in range(k):
        x,y,c = tuple(map(int,f.readline().split()))
        orks.append((x,y,c))
    for _ in range(l):
        x,y = tuple(map(int,f.readline().split()))
        RFs_initial_pos.append((x,y))
    for _ in range(l):
        x,y = tuple(map(int,f.readline().split()))
        RFs_goal_pos.append((x,y))
allRFs=dict(zip(RFs_initial_pos,RFs_goal_pos))

orks_map=[[0]*m for _ in range(n)]
for ork,(x,y,c) in enumerate(orks):
    for i in range(-c,c+1):
        for j in range(-c,c+1):
            if abs(i)+abs(j)<=c and x+i>=0 and x+i<n and y+j>=0 and y+j<m:
                    orks_map[x+i][y+j]=ork+1

The Helper functions to check the reaching the goal and check that the transion is valid or not

In [4]:
def goalTest(pos,pRFs):
    # print('goaltestcalled')
    if pRFs==l and pos==gandor:
        return True
    return False

def canGo(pos,ork,presence):
    x,y=pos
    if x<0 or x>=n or y<0 or y>=m:
        return False
    if k>0:
        if orks_map[x][y]==ork:
            if presence+1>orks[ork-1][2]:
                return False
    return True


The initial node (root) is where the gandalf starts the game

In [5]:
root=Node(gandalf_initial_pos)
actions={'pick':'pick','place':'place','R':(0,1),'D':(1,0),'U':(-1,0),'L':(0,-1)}

# BFS

In [6]:
def bfs():
    frontier=queue.Queue()
    explored=set()
    seen_states = 0
    frontier.put(root)
    explored.add(hash(root))
    while not frontier.empty():
        node=frontier.get()
        for action,transition in actions.items():
            if action == 'pick':
                if node.picked_RF is None and node.state in RFs_initial_pos and allRFs[node.state] not in node.placed_RFs:
                    child_with_RF=Node(node.state)
                    child_with_RF.parent=node
                    child_with_RF.path=copy.copy(node.path)
                    child_with_RF.picked_RF=allRFs[node.state]
                    child_with_RF.placed_RFs=copy.copy(node.placed_RFs)
                    child_with_RF.inOrk=node.inOrk
                    child_with_RF.presence=node.presence
                    seen_states+=1
                    child_hash=hash(child_with_RF)
                    frontier.put(child_with_RF)
                    explored.add(child_hash)
                    continue
            elif action == 'place':
                if node.state == node.picked_RF:
                    child=Node(node.state)
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=None
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.placed_RFs.add(node.state)
                    child.inOrk=node.inOrk
                    child.presence=node.presence

                    seen_states+=1
                    child_hash=hash(child)
                    frontier.put(child)
                    explored.add(hash(child))
                    continue

            else:
                if canGo((node.state[0]+transition[0],node.state[1]+transition[1]),node.inOrk,node.presence):
                    child=Node((node.state[0]+transition[0],node.state[1]+transition[1]))
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=node.picked_RF
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.inOrk=node.inOrk
                    child.presence=node.presence

                    map_ork=orks_map[child.state[0]][child.state[1]]
                    alan=map_ork
                    pedar=child.inOrk
                    if alan==pedar and alan>0:
                        child.presence+=1
                    
                    elif not alan==pedar and alan>0 and pedar is None:
                        child.presence=1
                        child.inOrk=map_ork
                    
                    elif pedar is None and alan>0:
                        child.presence=1
                        child.inOrk=map_ork
                    
                    else:
                        child.presence=0
                        child.inOrk=None

                    child.path+=action
                    child_hash=hash(child)
                    if child_hash not in explored:
                        seen_states+=1
                        if goalTest(child.state,len(child.placed_RFs)):
                            return seen_states, child.path
                        frontier.put(child)
                        explored.add(child_hash)

    return 0,''

## Test BFS

Here, we calculate the avrage execution time of BFS method for three times on four different testcases.

In [8]:
for test in range(4):
    times=[]
    test_name='test_0'+str(test)
    print('Results for',test_name)
    for _ in range(3):

        orks=[]
        RFs_initial_pos=[]
        RFs_goal_pos=[]
        with open('./tests/'+test_name+'.txt') as f:
            n,m = tuple(map(int,f.readline().split()))
            gandalf_initial_pos = tuple(map(int,f.readline().split()))
            gandor = tuple(map(int,f.readline().split()))
            k,l = tuple(map(int,f.readline().split()))
            for _ in range(k):
                x,y,c = tuple(map(int,f.readline().split()))
                orks.append((x,y,c))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_initial_pos.append((x,y))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_goal_pos.append((x,y))
        allRFs=dict(zip(RFs_initial_pos,RFs_goal_pos))

        orks_map=[[0]*m for _ in range(n)]
        for ork,(x,y,c) in enumerate(orks):
            for i in range(-c,c+1):
                for j in range(-c,c+1):
                    if abs(i)+abs(j)<=c and x+i>=0 and x+i<n and y+j>=0 and y+j<m:
                            orks_map[x+i][y+j]=ork+1

                            
        root=Node(gandalf_initial_pos)
        
        tic=time()
        a,b=bfs()
        toc=time()
        if len(times)==0:
            print('Seen Sates:',a)
            print('Path:',b)
            print('Path Length:',len(b))
            print('Time:',(toc-tic)*1000)
        times.append((toc-tic)*1000)

    print('\nMean Time:', (times[0]+times[1]+times[2])/3)
    print('------------------')

Results for test_00
Seen Sates: 10478
Path: RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR
Path Length: 48
Time: 213.69504928588867

Mean Time: 211.89403533935547
------------------
Results for test_01
Seen Sates: 1476
Path: RRRRDDDDDDDDLLLRRRUUURURUDLDLDDDLRRRRRUUUUDDDDLLLRRR
Path Length: 52
Time: 30.496835708618164

Mean Time: 26.5041987101237
------------------
Results for test_02
Seen Sates: 413
Path: RRRRRRRDDDDLLLLDLLLRDDRUURURRRDRDD
Path Length: 34
Time: 6.030797958374023

Mean Time: 5.938132603963216
------------------
Results for test_03
Seen Sates: 4504
Path: RRDDDDDDDDLLRDRRRRRRRRUUUUUUUUULLDDDDDLLDDDDLLLLRRRRUUUUURRRRDDDDD
Path Length: 66
Time: 79.4377326965332

Mean Time: 81.52993520100911
------------------


# IDS

In [9]:
root=Node(gandalf_initial_pos)
actions={'pick':'pick','place':'place','D':(1,0),'L':(0,-1),'U':(-1,0),'R':(0,1)}

In [10]:
def dls(search_depth):
    seen_states=0
    frontier=[]
    explored=set()
    frontier.append(root)
    explored.add(hash(root))

    while len(frontier)>0:
        node=frontier.pop()
        if node.depth+1> search_depth:
            continue

        for action,transition in actions.items():
            if action == 'pick':
                if node.picked_RF is None and node.state in RFs_initial_pos and allRFs[node.state] not in node.placed_RFs:
                    child_with_RF=Node(node.state)
                    child_with_RF.parent=node
                    child_with_RF.path=copy.deepcopy(node.path)
                    child_with_RF.picked_RF=allRFs[node.state]
                    child_with_RF.placed_RFs=copy.copy(node.placed_RFs)
                    child_with_RF.inOrk=node.inOrk
                    child_with_RF.presence=node.presence
                    child_with_RF.depth=node.depth+1
                    seen_states+=1
                    child_hash=hash(child_with_RF)
                    frontier.append(child_with_RF)
                    explored.add(child_hash)
                    continue
            elif action == 'place':
                if node.state == node.picked_RF:
                    child=Node(node.state)
                    child.parent=node
                    child.path=copy.deepcopy(node.path)
                    child.picked_RF=None
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.placed_RFs.add(node.state)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    child.depth=node.depth+1
                    seen_states+=1
                    child_hash=hash(child)
                    frontier.append(child)
                    explored.add(hash(child))
                    continue
            else:
                if canGo((node.state[0]+transition[0],node.state[1]+transition[1]),node.inOrk,node.presence):
                    child=Node((node.state[0]+transition[0],node.state[1]+transition[1]))
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=node.picked_RF
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    child.depth=node.depth+1
                    map_ork=orks_map[child.state[0]][child.state[1]]
                    alan=map_ork
                    pedar=child.inOrk
                    if alan==pedar and alan>0:
                        child.presence+=1
                    elif not alan==pedar and alan>0 and pedar is None:
                        child.presence=1
                        child.inOrk=map_ork
                    elif pedar is None and alan>0:
                        child.presence=1
                        child.inOrk=map_ork
                    else:
                        child.presence=0
                        child.inOrk=None
                    child.path+=action
                    child_hash=hash(child)
                    if child_hash not in explored:
                        seen_states+=1
                        if goalTest(child.state,len(child.placed_RFs)):
                            return seen_states, child.path
                        frontier.append(child)
                        explored.add(child_hash)
    return 0,''

In [11]:
def ids():
    search_depth=0

    while search_depth<100:
        search_depth+=1
        a,b=dls(search_depth)
        if a>0:
            return a,b
    
    return 0,''


## Test IDS

In [12]:
for test in range(4):
    times=[]
    test_name='test_0'+str(test)
    print('Results for',test_name)
    for _ in range(3):

        orks=[]
        RFs_initial_pos=[]
        RFs_goal_pos=[]
        with open('./tests/'+test_name+'.txt') as f:
            n,m = tuple(map(int,f.readline().split()))
            gandalf_initial_pos = tuple(map(int,f.readline().split()))
            gandor = tuple(map(int,f.readline().split()))
            k,l = tuple(map(int,f.readline().split()))
            for _ in range(k):
                x,y,c = tuple(map(int,f.readline().split()))
                orks.append((x,y,c))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_initial_pos.append((x,y))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_goal_pos.append((x,y))
        allRFs=dict(zip(RFs_initial_pos,RFs_goal_pos))

        orks_map=[[0]*m for _ in range(n)]
        for ork,(x,y,c) in enumerate(orks):
            for i in range(-c,c+1):
                for j in range(-c,c+1):
                    if abs(i)+abs(j)<=c and x+i>=0 and x+i<n and y+j>=0 and y+j<m:
                            orks_map[x+i][y+j]=ork+1

        root=Node(gandalf_initial_pos)

        tic=time()
        a,b=ids()
        toc=time()
        if len(times)==0:
            print('Seen Sates:',a)
            print('Path:',b)
            print('Path Length:',len(b))
            print('Time:',(toc-tic)*1000)
        times.append((toc-tic)*1000)

    print('\nMean Time:', (times[0]+times[1]+times[2])/3)
    print('------------------')

Results for test_00
Seen Sates: 56385
Path: RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR
Path Length: 48
Time: 6043.301343917847

Mean Time: 6045.726458231608
------------------
Results for test_01
Seen Sates: 19298
Path: RRRRDDLLDDDDDLDRRRUUURURULLDDDLDDRRRRRUUUUDDDLDLLRRR
Path Length: 52
Time: 3864.107608795166

Mean Time: 3942.2130584716797
------------------
Results for test_02
Seen Sates: 2507
Path: RRRRRRRLLLLDDDDLDLLRDDRUURURRRDRDD
Path Length: 34
Time: 437.61730194091797

Mean Time: 429.79780832926434
------------------
Results for test_03
Seen Sates: 71664
Path: RRDDDDDLDLDDRDRRRRRRRRUUUUUUUUULLLLLLLDDDDDDDDLDRRRRUUUUURRRRDDDDD
Path Length: 66
Time: 20312.01219558716

Mean Time: 20465.33226966858
------------------


# A*

In [13]:
class PriorityQueue(object):
    def __init__(self):
        self.queue = []

    def __str__(self):
        return ' '.join([str(i) for i in self.queue])

    def empty(self):
        return len(self.queue) == 0

    def put(self, data):
        self.queue.append(data)

    def get(self):
        max = 0
        for i in range(len(self.queue)):
            if self.queue[i].h > self.queue[max].h:
                max = i
        item = self.queue[max]
        del self.queue[max]
        return item

In [14]:
def heuristic(pos,picked_RF,placed_RFs):
    hn=abs(pos[0]-gandor[0])+abs(pos[1]-gandor[1])
    if len(placed_RFs)==0:
        return hn

    for RF_pos, RF_goal in allRFs.items():
        if RF_goal not in placed_RFs and not RF_pos==picked_RF:
            hn-=2*abs(pos[0]-RF_pos[0])+abs(pos[1]-RF_pos[1])

    return hn


In [15]:
root=Node(gandalf_initial_pos)
root.h=heuristic(gandalf_initial_pos,None,set())
actions={'pick':'pick','place':'place','D':(1,0),'L':(0,-1),'U':(-1,0),'R':(0,1)}

In [16]:
def a_star():
    frontier=PriorityQueue()
    explored=set()
    seen_states = 0
    frontier.put(root)
    explored.add(hash(root))
    while not frontier.empty():
        node=frontier.get()
        for action,transition in actions.items():
            if action == 'pick':
                if node.picked_RF is None and node.state in RFs_initial_pos and allRFs[node.state] not in node.placed_RFs:
                    child=Node(node.state)
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=allRFs[node.state]
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    child.h=heuristic(child.state,child.picked_RF,child.placed_RFs)+len(child.path)
                    seen_states+=1
                    child_hash=hash(child)
                    frontier.put(child)
                    explored.add(child_hash)
                    continue
            elif action == 'place':
                if node.state == node.picked_RF:
                    child=Node(node.state)
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=None
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.placed_RFs.add(node.state)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    child.h=heuristic(child.state,child.picked_RF,child.placed_RFs)+len(child.path)
                    seen_states+=1
                    child_hash=hash(child)
                    frontier.put(child)
                    explored.add(hash(child))
                    continue
            else:
                if canGo((node.state[0]+transition[0],node.state[1]+transition[1]),node.inOrk,node.presence):
                    child=Node((node.state[0]+transition[0],node.state[1]+transition[1]))
                    child.parent=node
                    child.path=copy.copy(node.path)
                    child.picked_RF=node.picked_RF
                    child.placed_RFs=copy.copy(node.placed_RFs)
                    child.inOrk=node.inOrk
                    child.presence=node.presence
                    child.h=heuristic(child.state,child.picked_RF,child.placed_RFs)+len(child.path)
                    map_ork=orks_map[child.state[0]][child.state[1]]
                    alan=map_ork
                    pedar=child.inOrk
                    if alan==pedar and alan>0:
                        child.presence+=1
                    elif not alan==pedar and alan>0 and pedar is None:
                        child.presence=1
                        child.inOrk=map_ork
                    elif pedar is None and alan>0:
                        child.presence=1
                        child.inOrk=map_ork
                    else:
                        child.presence=0
                        child.inOrk=None
                    child.path+=action
                    child_hash=hash(child)
                    if child_hash not in explored:
                        seen_states+=1
                        if goalTest(child.state,len(child.placed_RFs)):
                            return seen_states, child.path
                        frontier.put(child)
                        explored.add(child_hash)
    return 0,''


In [17]:
tic=time()
a,b=a_star()
toc=time()
print(a,b,len(b))
print('time',(toc-tic)*1000)

957 DDRDRDDLDLDDDRLUUUURRUULULUURRRRRRRRRLLLLLLDLDDRDRRURRDRRULLLUULULLLDDDUUURRRDRDDLDLLDLLDLDDUUURRUUUUURRRRRLLLLLDDDRDRDDDLDLLDLUUUURRUULULUURRRRRDRDDLDLDDDLDDRRRRRR 164
time 80.89780807495117


In [18]:
for test in range(4):
    times=[]
    test_name='test_0'+str(test)
    print('Results for',test_name)
    for _ in range(3):

        orks=[]
        RFs_initial_pos=[]
        RFs_goal_pos=[]
        with open('./tests/'+test_name+'.txt') as f:
            n,m = tuple(map(int,f.readline().split()))
            gandalf_initial_pos = tuple(map(int,f.readline().split()))
            gandor = tuple(map(int,f.readline().split()))
            k,l = tuple(map(int,f.readline().split()))
            for _ in range(k):
                x,y,c = tuple(map(int,f.readline().split()))
                orks.append((x,y,c))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_initial_pos.append((x,y))
            for _ in range(l):
                x,y = tuple(map(int,f.readline().split()))
                RFs_goal_pos.append((x,y))
        allRFs=dict(zip(RFs_initial_pos,RFs_goal_pos))

        orks_map=[[0]*m for _ in range(n)]
        for ork,(x,y,c) in enumerate(orks):
            for i in range(-c,c+1):
                for j in range(-c,c+1):
                    if abs(i)+abs(j)<=c and x+i>=0 and x+i<n and y+j>=0 and y+j<m:
                            orks_map[x+i][y+j]=ork+1

        root=Node(gandalf_initial_pos)

        tic=time()
        a,b=a_star()
        toc=time()
        print('Seen Sates:',a)
        print('Path:',b)
        print('Path Length:',len(b))
        print('Time:',(toc-tic)*1000)
        times.append((toc-tic)*1000)

    print('\nMean Time:', (times[0]+times[1]+times[2])/3)
    print('------------------')

Results for test_00
Seen Sates: 2042
Path: LDDRRURUURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUUDULDURRDULLLDURRRRDDLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLUUURRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRDDRRUURURRDDL
Path Length: 180
Time: 96.30799293518066
Seen Sates: 2042
Path: LDDRRURUURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUUDULDURRDULLLDURRRRDDLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLUUURRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRDDRRUURURRDDL
Path Length: 180
Time: 68.75896453857422
Seen Sates: 2042
Path: LDDRRURUURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUURURRDDDRRUUDULDURRDULLLDURRRRDDLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLUUURRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRDDRRUURURRDDL
Path Length: 180
Time: 69.0913200378418

Mean Time: 78.05275917053223
------------------
Results for test_01
Seen Sates: 676
Path: DDDDRRRRRRRRLLLLLLLLDDDRRURRDDRULUUUUURRDLLULLDDDDRDDUULUUUURRLLULLDDRDRRRDDLDLLDLUUUUUUUURRDDDDDDDRDRRRRR
Path Length: 106
Time: 18.718242645263672
Seen S

In [380]:
aa=Node((0,0))
a=Node((1,0))
a.parent=aa
b=Node((2,0))
b.parent=a
c=Node((3,0))
c.parent=b
d=Node((4,0))
d.parent=c

print('bb:',canGo(c.state,1,1))

bb: True
